In [ ]:
import pandas as pd

# Load into a DataFrame
df = pd.read_csv('ztf_image_search_results_full.csv')

df.head()

,ra,dec,infobits,field,ccdid,qid,rcid,fid,filtercode,pid,...,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,ipac_pub_date,ipac_gid
0,142.513076,22.239831,67108864,570,16,1,60,2,zr,769412526015,...,142.972615,22.679692,142.036715,22.664666,142.055997,21.798581,142.986538,21.813558,2020-12-09 00:00:00+00,2
1,141.637575,21.360661,0,570,16,3,62,2,zr,1503405246215,...,142.091906,21.803041,141.161603,21.782992,141.186132,20.917351,142.110832,20.937013,2022-09-07 00:00:00+00,2
2,141.618681,22.225694,0,570,16,2,61,2,zr,1504338726115,...,142.075514,22.668183,141.139773,22.647755,141.164682,21.781999,142.094901,21.802092,2022-09-07 00:00:00+00,2
3,141.596498,22.204009,67108912,570,16,2,61,1,zg,1066544346115,...,142.053404,22.646507,141.117659,22.626244,141.142411,21.760423,142.072649,21.780266,2020-06-24 00:00:00+00,1
4,141.637182,19.443859,0,570,12,3,46,2,zr,1521276364615,...,142.085851,19.886330,141.167231,19.866253,141.191126,19.000577,142.104704,19.020278,2022-09-07 00:00:00+00,3


In [ ]:
# Summary of the raw dataset
print('Rows, Columns:', df.shape)

Rows, Columns: (62368, 44)

Column types:
float64    25
int64      14
object      5
Name: count, dtype: int64

Missing values per column:
ra               0
dec              0
infobits         0
field            0
ccdid            0
qid              0
rcid             0
fid              0
filtercode       0
pid              0
nid              0
expid            0
itid             0
imgtype          0
imgtypecode      0
obsdate          0
obsjd            0
exptime          0
filefracday      0
seeing           0
airmass          0
moonillf         0
moonesb          0
maglimit         0
crpix1           0
crpix2           0
crval1           0
crval2           0
cd11             0
cd12             0
cd21             0
cd22             0
ra1              0
dec1             0
ra2              0
dec2             0
ra3              0
dec3             0
ra4              0
dec4             0
ipac_pub_date    0
ipac_gid         0
log1p_exptime    0
log1p_airmass    0
dtype: int64


In [13]:
print('\nColumn types:')
print(df.dtypes.value_counts())


Column types:
float64    25
int64      14
object      5
Name: count, dtype: int64


In [12]:
print('\nMissing values per column:')
print(df.isnull().sum())


Missing values per column:
ra               0
dec              0
infobits         0
field            0
ccdid            0
qid              0
rcid             0
fid              0
filtercode       0
pid              0
nid              0
expid            0
itid             0
imgtype          0
imgtypecode      0
obsdate          0
obsjd            0
exptime          0
filefracday      0
seeing           0
airmass          0
moonillf         0
moonesb          0
maglimit         0
crpix1           0
crpix2           0
crval1           0
crval2           0
cd11             0
cd12             0
cd21             0
cd22             0
ra1              0
dec1             0
ra2              0
dec2             0
ra3              0
dec3             0
ra4              0
dec4             0
ipac_pub_date    0
ipac_gid         0
log1p_exptime    0
log1p_airmass    0
dtype: int64


In [ ]:
# Replace common text placeholders with NaN for consistent handling
import numpy as np
df.replace(['', ' ', 'NA', 'NaN', 'nan', 'None', 'none', 'NULL'], np.nan, inplace=True)
print('\nAfter normalization of placeholders, missing per column:')
print(df.isnull().sum())

In [ ]:
# 2) Remove duplicate rows
before = df.shape[0]
df = df.drop_duplicates().reset_index(drop=True)
after = df.shape[0]
print(f'Removed {before - after} duplicate rows')

In [ ]:
# 3) Remove nulls with strategy: drop rows with >50% missing, impute numerics with median and categoricals with mode
threshold = int(df.shape[1] * 0.5)
df = df.dropna(thresh=threshold).reset_index(drop=True)
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for c in num_cols:
    if df[c].isnull().any():
        df[c] = df[c].fillna(df[c].median())
for c in cat_cols:
    if df[c].isnull().any():
        mode = df[c].mode()
        fill_val = mode.iloc[0] if not mode.empty else 'Unknown'
        df[c] = df[c].fillna(fill_val)
print('After null handling, remaining missing per column:')
print(df.isnull().sum())

Rows, Columns: (82955, 42)

Column types:
float64    23
int64      14
object      5
Name: count, dtype: int64

Missing values per column:
ra               0
dec              0
infobits         0
field            0
ccdid            0
qid              0
rcid             0
fid              0
filtercode       0
pid              0
nid              0
expid            0
itid             0
imgtype          0
imgtypecode      0
obsdate          0
obsjd            0
exptime          0
filefracday      0
seeing           0
airmass          0
moonillf         0
moonesb          0
maglimit         0
crpix1           0
crpix2           0
crval1           0
crval2           0
cd11             0
cd12             0
cd21             0
cd22             0
ra1              0
dec1             0
ra2              0
dec2             0
ra3              0
dec3             0
ra4              0
dec4             0
ipac_pub_date    0
ipac_gid         0
dtype: int64

After normalization of placeholders, missing per c

In [5]:
# 4) Remove outliers using IQR on numeric columns (iterative cumulative removal)
import pandas as pd
num_cols = df.select_dtypes(include=['number']).columns.tolist()
print('Numeric columns to check for outliers:', num_cols)
initial_rows = df.shape[0]
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    if pd.isnull(IQR) or IQR == 0:
        continue
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower) & (df[col] <= upper)].reset_index(drop=True)
print(f'Removed {initial_rows - df.shape[0]} rows as outliers (cumulative)')

Numeric columns to check for outliers: ['ra', 'dec', 'infobits', 'field', 'ccdid', 'qid', 'rcid', 'fid', 'pid', 'nid', 'expid', 'itid', 'obsjd', 'exptime', 'filefracday', 'seeing', 'airmass', 'moonillf', 'moonesb', 'maglimit', 'crpix1', 'crpix2', 'crval1', 'crval2', 'cd11', 'cd12', 'cd21', 'cd22', 'ra1', 'dec1', 'ra2', 'dec2', 'ra3', 'dec3', 'ra4', 'dec4', 'ipac_gid']
Removed 20587 rows as outliers (cumulative)


In [6]:
# 5) Remove incorrect / impossible values (example astronomical checks)
import numpy as np
if 'ra' in df.columns:
    before = df.shape[0]
    df = df[(df['ra'] >= 0) & (df['ra'] <= 360)].reset_index(drop=True)
    print('Filtered RA outside [0,360], removed', before - df.shape[0])
if 'dec' in df.columns:
    before = df.shape[0]
    df = df[(df['dec'] >= -90) & (df['dec'] <= 90)].reset_index(drop=True)
    print('Filtered Dec outside [-90,90], removed', before - df.shape[0])
# Replace infinities and drop any remaining NaNs
df = df.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
print('After removing infinite/NaN values, rows:', df.shape[0])
# Example: require positive flux_err if those columns exist
if set(['flux', 'flux_err']).issubset(df.columns):
    before = df.shape[0]
    df = df[df['flux_err'] > 0].reset_index(drop=True)
    print('Removed rows with non-positive flux_err:', before - df.shape[0])

Filtered RA outside [0,360], removed 0
Filtered Dec outside [-90,90], removed 0
After removing infinite/NaN values, rows: 62368


In [7]:
# 6) Scaling: Standardization (store standardized copy)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
exclude_patterns = ['id', 'Id', 'ID', 'name', 'filename', 'time', 'date', 'jd']
exclude_cols = [c for c in df.columns for p in exclude_patterns if p in c]
cols_to_scale = [c for c in numeric_cols if c not in exclude_cols]
print('Columns to scale:', cols_to_scale)
scaler = StandardScaler()
if cols_to_scale:
    df_standardized = df.copy()
    df_standardized[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    print('Standardization complete. Dataframe `df_standardized` is available.')
else:
    print('No numeric columns found to scale.')

Columns to scale: ['ra', 'dec', 'infobits', 'field', 'filefracday', 'seeing', 'airmass', 'moonillf', 'moonesb', 'maglimit', 'crpix1', 'crpix2', 'crval1', 'crval2', 'cd11', 'cd12', 'cd21', 'cd22', 'ra1', 'dec1', 'ra2', 'dec2', 'ra3', 'dec3', 'ra4', 'dec4']
Standardization complete. Dataframe `df_standardized` is available.


In [8]:
# 7) Feature engineering
import numpy as np
if set(['flux', 'flux_err']).issubset(df.columns):
    df['snr'] = df['flux'] / df['flux_err']
    print('Created `snr` feature')
possible_date_cols = [c for c in df.columns if any(x in c.lower() for x in ['date','time','jd','mjd'])]
if possible_date_cols:
    date_col = possible_date_cols[0]
    try:
        df[date_col] = pd.to_datetime(df[date_col])
        df['year'] = df[date_col].dt.year
        df['month'] = df[date_col].dt.month
        df['day'] = df[date_col].dt.day
        print(f'Derived year/month/day from {date_col}')
    except Exception as e:
        print('Could not parse date column', date_col, ' - ', e)
for c in df.select_dtypes(include=['number']).columns:
    if (df[c] > 0).all() and df[c].skew() > 1:
        df['log1p_' + c] = np.log1p(df[c])
        print('Created log1p_' + c)
print('Feature engineering complete.')

Could not parse date column obsdate  -  time data "2018-03-25 06:35:35+00" doesn't match format "%Y-%m-%d %H:%M:%S.%f%z", at position 35. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
Created log1p_exptime
Created log1p_airmass
Feature engineering complete.


In [9]:
# 8) Handle class imbalance (try SMOTE if a target column exists)
from collections import Counter
possible_targets = [c for c in df.columns if c.lower() in ['label','class','target','type']]
if possible_targets:
    target = possible_targets[0]
    print('Target column detected:', target)
    print('Class counts before:', Counter(df[target]))
    try:
        from imblearn.over_sampling import SMOTE
        features = df.drop(columns=[target]).select_dtypes(include=['number']).columns.tolist()
        if features:
            X = df[features]
            y = df[target]
            sm = SMOTE(random_state=42)
            X_res, y_res = sm.fit_resample(X, y)
            df_res = pd.DataFrame(X_res, columns=features)
            df_res[target] = y_res
            print('Resampled dataset shape:', df_res.shape)
        else:
            print('No numeric feature columns available for SMOTE; skipping resampling.')
    except Exception as e:
        print('SMOTE failed or imblearn not installed:', e)
else:
    print('No obvious target column found; skipping imbalance handling')

No obvious target column found; skipping imbalance handling


In [10]:
# 9) Final checks and save cleaned data
print('Final shape:', df.shape)
print(df.dtypes)
output_path = 'ztf_image_search_results_full_cleaned.csv'
df.to_csv(output_path, index=False)
print('Cleaned dataset saved to', output_path)
try:
    df_standardized.to_csv('ztf_image_search_results_full_standardized.csv', index=False)
    print('Standardized dataset saved to ztf_image_search_results_full_standardized.csv')
except NameError:
    pass
df.head()

Final shape: (62368, 44)
ra               float64
dec              float64
infobits           int64
field              int64
ccdid              int64
qid                int64
rcid               int64
fid                int64
filtercode        object
pid                int64
nid                int64
expid              int64
itid               int64
imgtype           object
imgtypecode       object
obsdate           object
obsjd            float64
exptime            int64
filefracday        int64
seeing           float64
airmass          float64
moonillf         float64
moonesb            int64
maglimit         float64
crpix1           float64
crpix2           float64
crval1           float64
crval2           float64
cd11             float64
cd12             float64
cd21             float64
cd22             float64
ra1              float64
dec1             float64
ra2              float64
dec2             float64
ra3              float64
dec3             float64
ra4              float64


,ra,dec,infobits,field,ccdid,qid,rcid,fid,filtercode,pid,...,ra2,dec2,ra3,dec3,ra4,dec4,ipac_pub_date,ipac_gid,log1p_exptime,log1p_airmass
0,142.513076,22.239831,67108864,570,16,1,60,2,zr,769412526015,...,142.036715,22.664666,142.055997,21.798581,142.986538,21.813558,2020-12-09 00:00:00+00,2,3.433987,0.752830
1,141.637575,21.360661,0,570,16,3,62,2,zr,1503405246215,...,141.161603,21.782992,141.186132,20.917351,142.110832,20.937013,2022-09-07 00:00:00+00,2,3.433987,0.755183
2,141.618681,22.225694,0,570,16,2,61,2,zr,1504338726115,...,141.139773,22.647755,141.164682,21.781999,142.094901,21.802092,2022-09-07 00:00:00+00,2,3.433987,0.710004
3,141.637182,19.443859,0,570,12,3,46,2,zr,1521276364615,...,141.167231,19.866253,141.191126,19.000577,142.104704,19.020278,2022-09-07 00:00:00+00,3,3.433987,0.709513
4,141.637223,19.440343,0,570,12,3,46,1,zg,1519302464615,...,141.167275,19.862742,141.191122,18.996967,142.104734,19.016760,2021-06-30 00:00:00+00,1,3.433987,0.710496
